In [7]:
import re  # 정규 표현식 모듈 추가
import pandas as pd  # CSV 저장을 위한 pandas 추가
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동 (테스트 URL)
driver.get('https://map.naver.com/p/search/%EB%B1%85%EB%B1%85%EB%A7%89%EA%B5%AD%EC%88%98/place/33964233?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 가져오기
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 가져오기
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 가져오기 (부모 요소의 텍스트 가져오기)
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
    )
    rating_text = rating_element.text
    rating = rating_text.replace("별점", "").strip()

    # 방문자 리뷰 가져오기
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 가져오기
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '사진' 탭 클릭
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='사진']"))
    )
    review_tab_element.click()
    print("사진 탭 클릭 완료")
    
    # 사진이 로딩될 때까지 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "wzrbN"))
    )
    print("사진 로딩 완료")
    
    # 다섯 개의 사진 URL 가져오기
    photo_elements = driver.find_elements(By.CLASS_NAME, "wzrbN")[:5]
    photo_urls = [photo.find_element(By.TAG_NAME, "img").get_attribute("src") for photo in photo_elements]
    
    # 결과 출력
    for i, url in enumerate(photo_urls, 1):
        print(f"사진 {i} URL:", url)

except Exception as e:
    print("오류 발생:", e)
# finally:
#     # 드라이버 닫기
#     driver.quit()


가게 이름: 뱅뱅막국수
업종 카테고리: 국수
별점: 4.69
방문자 리뷰: 3,881
블로그 리뷰: 1,955
사진 탭 클릭 완료
사진 로딩 완료
사진 1 URL: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240814_240%2F1723564578931L49UB_JPEG%2FKakaoTalk_20240811_143205935_02.jpg
사진 2 URL: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240814_173%2F17235639811520TGyO_JPEG%2FKakaoTalk_20240813_112241814.jpg
사진 3 URL: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240814_5%2F1723564492502CaLUq_JPEG%2FKakaoTalk_20240813_135430257.jpg
사진 4 URL: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fvideo-phinf.pstatic.net%2F20241104_198%2F1730721926449uwxet_JPEG%2F02oTiI3R0p_03.jpg
사진 5 URL: https://pup-review-phinf.pstatic.net/MjAyNDExMDVfMyAg/MDAxNzMwNzgyODczODM2.yUJGMMXSG1ZHH1nkpH7gNRmNbjdOw2GhiuZ8xSiTkBAg.qe8Yr27zWDqZ1g2PNclm

In [17]:
import re
import pandas as pd
from IPython.display import Image, display
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EB%B1%85%EB%B1%85%EB%A7%89%EA%B5%AD%EC%88%98/place/33964233?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # '사진' 탭 클릭
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='사진']"))
    )
    review_tab_element.click()
    print("사진 탭 클릭 완료")
    
    # 사진이 로딩될 때까지 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "wzrbN"))
    )
    print("사진 로딩 완료")
    
    # 다섯 개의 사진 URL 가져오기
    photo_elements = driver.find_elements(By.CLASS_NAME, "wzrbN")[:5]
    photo_urls = [photo.find_element(By.TAG_NAME, "img").get_attribute("src") for photo in photo_elements]
    
    # URL 5개 확인
    for i, url in enumerate(photo_urls, 1):
        print(f"사진 {i} URL:", url)

    # Jupyter Notebook에서 이미지를 표시
    for i, url in enumerate(photo_urls, 1):
        print(f"이미지 {i} 표시 중...")
        display(Image(url=url))
        time.sleep(1)  # 각 이미지 표시 후 1초 대기

except Exception as e:
    print("오류 발생:", e)

finally:
    # 드라이버 닫기
    driver.quit()



사진 탭 클릭 완료
사진 로딩 완료
사진 1 URL: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240814_240%2F1723564578931L49UB_JPEG%2FKakaoTalk_20240811_143205935_02.jpg
사진 2 URL: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240814_173%2F17235639811520TGyO_JPEG%2FKakaoTalk_20240813_112241814.jpg
사진 3 URL: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240814_5%2F1723564492502CaLUq_JPEG%2FKakaoTalk_20240813_135430257.jpg
사진 4 URL: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fvideo-phinf.pstatic.net%2F20241104_198%2F1730721926449uwxet_JPEG%2F02oTiI3R0p_03.jpg
사진 5 URL: https://pup-review-phinf.pstatic.net/MjAyNDExMDVfMyAg/MDAxNzMwNzgyODczODM2.yUJGMMXSG1ZHH1nkpH7gNRmNbjdOw2GhiuZ8xSiTkBAg.qe8Yr27zWDqZ1g2PNclmLDYX8NeJdFEjJIbWpYCtyOAg.JPEG/7D6DCEBB-35EC-4D49-85DC-BF092A42

이미지 2 표시 중...


이미지 3 표시 중...


이미지 4 표시 중...


이미지 5 표시 중...
